In [3]:
# 载入需要用的库
import cifar10, cifar10_input
import tensorflow as tf
import numpy as np
import math
import time


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [4]:
batch_size = 128
data = './cifar10_data/cifar-10-batches-bin'
images_train, labels_train = cifar10_input.inputs(eval_data = False, batch_size = batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data = True, batch_size = batch_size)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


W0614 09:41:31.488114 4752627136 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow_datasets/core/file_format_adapter.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset cifar10 downloaded and prepared to /Users/srn/tensorflow_datasets/cifar10/1.0.2. Subsequent calls will reuse this data.
Instructions for updating:
Colocations handled automatically by placer.


W0614 09:41:38.761754 4752627136 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py:423: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


W0614 09:41:38.920570 4752627136 deprecation.py:323] From /anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/image_ops_impl.py:1241: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [5]:
def weight_variable(shape, stddev):
    initial = tf.truncated_normal(shape, stddev=stddev)  # stddev=stddev！！！
    return tf.Variable(initial)


def bias_variable(cons, shape):
    initial = tf.constant(cons, shape=shape)  # 必须是 shape=shape
    return tf.Variable(initial)


def conv(x, W):
    return tf.nn.conv2d(x, W, [1, 1, 1, 1], padding='SAME')


def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [6]:
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

In [7]:
# 第一层
weight1 = weight_variable([5, 5, 3, 64], 5e-2)
bias1 = bias_variable(0.0, [64])

conv1 = tf.nn.relu(conv(image_holder, weight1) + bias1)
pool1 = max_pool_3x3(conv1)

# 第二层
weight2 = weight_variable([5, 5, 64, 64], 5e-2)
bias2 = bias_variable(0.1, [64])

conv2 = tf.nn.relu(conv(pool1, weight2) + bias2)
pool2 = max_pool_3x3(conv2)

reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value

# 全连接层
weight3 = weight_variable([dim, 384], 0.04)
bias3 = bias_variable(0.1, [384])

local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

# 全连接层
weight4 = weight_variable([384, 192], 0.04)
bias4 = bias_variable(0.1, [192])

local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

# 输出
weight5 = weight_variable([192, 10], 1 / 192.0)
bias5 = bias_variable(0.0, [10])
logits = tf.matmul(local4, weight5) + bias5

In [8]:
# 损失函数
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels,
                                                                   name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)
    return tf.add_n(tf.get_collection('losses'), name='total_loss')


loss = loss(logits, label_holder)
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

In [9]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
tf.train.start_queue_runners()

max_steps = 3000
for step in range(max_steps):
    start_time = time.time()
    image_batch, label_batch = sess.run([images_train, labels_train])
    _, loss_value = sess.run([train_op, loss], feed_dict={image_holder: image_batch, label_holder: label_batch})
    duration = time.time() - start_time

    if step % 10 == 0:
        examples_per_sec = batch_size / duration
        sec_per_batch = float(duration)

        print('step {},loss={},({} examples/sec; {} sec/batch)'.format(step, loss_value, examples_per_sec, sec_per_batch))

Instructions for updating:
To construct input pipelines, use the `tf.data` module.


W0614 09:42:14.357114 4752627136 deprecation.py:323] From <ipython-input-9-02b76c52026f>:3: start_queue_runners (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.


W0614 09:42:14.359175 4752627136 queue_runner_impl.py:471] `tf.train.start_queue_runners()` was called when no queue runners were defined. You can safely remove the call to this deprecated function.


step 0,loss=2.3059282302856445,(123.40075699398868 examples/sec; 1.0372707843780518 sec/batch)
step 10,loss=1.996286153793335,(461.15363544298145 examples/sec; 0.27756476402282715 sec/batch)
step 20,loss=1.8116545677185059,(485.46633788081425 examples/sec; 0.26366400718688965 sec/batch)
step 30,loss=1.8194639682769775,(462.2323977979705 examples/sec; 0.2769169807434082 sec/batch)
step 40,loss=1.7946010828018188,(337.3631289690544 examples/sec; 0.3794131278991699 sec/batch)
step 50,loss=1.830093264579773,(419.78196739769555 examples/sec; 0.3049201965332031 sec/batch)
step 60,loss=1.6019084453582764,(424.34182510858096 examples/sec; 0.30164361000061035 sec/batch)
step 70,loss=1.6338410377502441,(436.39242819333987 examples/sec; 0.29331398010253906 sec/batch)
step 80,loss=1.6329753398895264,(436.9660525152751 examples/sec; 0.29292893409729004 sec/batch)
step 90,loss=1.3785829544067383,(436.3374536635558 examples/sec; 0.2933509349822998 sec/batch)
step 100,loss=1.5947816371917725,(425.8365

step 850,loss=0.8812328577041626,(464.7717932558123 examples/sec; 0.2754039764404297 sec/batch)
step 860,loss=0.992067813873291,(432.5101140667257 examples/sec; 0.2959468364715576 sec/batch)
step 870,loss=0.8043610453605652,(435.563961356106 examples/sec; 0.2938718795776367 sec/batch)
step 880,loss=1.0125097036361694,(441.65350738281643 examples/sec; 0.28981995582580566 sec/batch)
step 890,loss=1.0449076890945435,(459.30998987050674 examples/sec; 0.27867889404296875 sec/batch)
step 900,loss=0.8437172770500183,(464.0438674434283 examples/sec; 0.2758359909057617 sec/batch)
step 910,loss=0.8402470350265503,(455.03974884581123 examples/sec; 0.2812941074371338 sec/batch)
step 920,loss=0.9042041897773743,(448.25307256599336 examples/sec; 0.285552978515625 sec/batch)
step 930,loss=0.8791593909263611,(465.38700295856194 examples/sec; 0.2750399112701416 sec/batch)
step 940,loss=0.8509799242019653,(410.860743645433 examples/sec; 0.3115410804748535 sec/batch)
step 950,loss=0.9665051698684692,(412

step 1690,loss=0.6561191082000732,(439.96688555570034 examples/sec; 0.29093098640441895 sec/batch)
step 1700,loss=0.6930822134017944,(410.61532307593467 examples/sec; 0.31172728538513184 sec/batch)
step 1710,loss=0.6222703456878662,(355.6234889819635 examples/sec; 0.35993123054504395 sec/batch)
step 1720,loss=0.7361242771148682,(461.40054728215284 examples/sec; 0.2774162292480469 sec/batch)
step 1730,loss=0.66460120677948,(325.76940538552884 examples/sec; 0.3929159641265869 sec/batch)
step 1740,loss=0.6174150109291077,(409.25716219780486 examples/sec; 0.3127617835998535 sec/batch)
step 1750,loss=0.5816913843154907,(432.5606431475427 examples/sec; 0.2959122657775879 sec/batch)
step 1760,loss=0.5830256938934326,(380.07483823466947 examples/sec; 0.3367757797241211 sec/batch)
step 1770,loss=0.6703760623931885,(386.8846366985977 examples/sec; 0.33084797859191895 sec/batch)
step 1780,loss=0.8538064956665039,(445.81276344153883 examples/sec; 0.28711605072021484 sec/batch)
step 1790,loss=0.599

step 2530,loss=0.7015682458877563,(462.3677473917739 examples/sec; 0.27683591842651367 sec/batch)
step 2540,loss=0.40769144892692566,(437.39182703470726 examples/sec; 0.29264378547668457 sec/batch)
step 2550,loss=0.5442003011703491,(356.31290866373763 examples/sec; 0.3592348098754883 sec/batch)
step 2560,loss=0.4701751172542572,(460.7301814786559 examples/sec; 0.2778198719024658 sec/batch)
step 2570,loss=0.503272533416748,(439.4004645505989 examples/sec; 0.2913060188293457 sec/batch)
step 2580,loss=0.3086070418357849,(401.07016007817145 examples/sec; 0.31914615631103516 sec/batch)
step 2590,loss=0.5221357345581055,(310.79943984758484 examples/sec; 0.41184115409851074 sec/batch)
step 2600,loss=0.31357908248901367,(446.8275621568401 examples/sec; 0.28646397590637207 sec/batch)
step 2610,loss=0.4729779362678528,(446.74799268721534 examples/sec; 0.2865149974822998 sec/batch)
step 2620,loss=0.35208484530448914,(438.1972231041781 examples/sec; 0.29210591316223145 sec/batch)
step 2630,loss=0.

In [10]:
num_examples = 10000
num_iter = int(math.ceil(num_examples / batch_size))  # 计算一共有多少组
true_count = 0
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch, label_batch = sess.run([images_test, labels_test])
    predictions = sess.run([top_k_op], feed_dict={image_holder: image_batch, label_holder: label_batch})
    true_count += np.sum(predictions)
    step += 1

precision = true_count / total_sample_count
print('precision = ', precision)

precision =  0.8302017405063291
